In [ ]:
# load the model from the file "sbm3x3_pclam_roc_0.210_auc_0.860"
import torch
from torch_geometric.transforms import TwoHop

import sys
if '..' not in sys.path:
    sys.path.insert(0, '..')

from datasets.import_dataset import import_dataset
from utils.plotting import *
from trainer import Trainer
import optimization_utils as ou
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Using device:', device)
cpu = torch.device("cpu")

%load_ext autoreload
%autoreload 2


In [ ]:
%who

### Run anomaly detection experiment with global params from hypers/hypers_link_prediction.yaml. 

In [ ]:

model_names = ['pieclam']
ds_names = ['elliptic']
densifiable_ds = ['photo','reddit']
num_reps = 2


config_triplets = [['feat_opt', 'n_iter', 2200]]

for model_name in model_names:    
    for ds_name in ds_names:
        #todo: make a data collection scheme
        
        ''' ds_to_use chooses whether to use the original dataset or the densified one. The trainer ds is the one that is used by the trainer that modifies it.'''
        ds = import_dataset(ds_name)
        ds_to_use = ds
        
        if ds_name in densifiable_ds:
            fat_ds = TwoHop()(ds)
            fat_ds.edge_attr = torch.ones(fat_ds.edge_index.shape[1]).bool()
            ds_to_use = fat_ds
    

        losseses = []
        acc_testses = []
        acc_valses = []
        try:
            for i in range(num_reps):
                trainer_anomaly = Trainer(
                    model_name=model_name,
                    device=device,
                    dataset=ds_to_use,
                    attr_opt=True,
                    task='anomaly_unsupervised',
                    use_global_config_base=True,  
                    config_triplets_to_change=config_triplets          
                )
                
                losses, acc_test, acc_val = trainer_anomaly.train(
                    init_type='small_gaus',
                    init_feats=True,
                    acc_every=20,
                    plot_every=-1,
                    verbose=False,
                    verbose_in_funcs=False
                )
                losseses.append(losses)
                acc_testses.append(acc_test)
                acc_valses.append(acc_val)
                

        except (Exception, KeyboardInterrupt) as e:
            raise e
        finally:
            if trainer_anomaly is not None:
                del trainer_anomaly
            del ds
            # del ds_to_use
            if ds_name in densifiable_ds:
                del fat_ds
            torch.cuda.empty_cache()



### Scan ranges for hyper parameters

In [ ]:
model_name = 'ieclam'
ds_names = ['photo', 'reddit', 'elliptic']
densifiable_ds = ['photo', 'reddit']
n_reps = 1
use_global_config_base = True

#* perturb the global config
deltas ={
'clamiter_init': {'s_reg': 0.001,
                  'l1_reg': 0.001,
                  'dim_feat': 2},
'feat_opt': {'n_iter': 200,
            'lr': 0.0000005},
'prior_opt': {'n_iter': 200,
              'lr': 0.0000005,
              'noise_amp': 0.01},
'back_forth': {'sceduler_gamma' : 0.1}
}

range_triplets = ou.perturb_config('anomaly_unsupervised', 
                                   model_name, 
                                   deltas, 
                                   use_global_config=use_global_config_base)



ou.multi_ds_anomaly(
    model_name,
    range_triplets,
    n_reps,
    use_global_config_base=use_global_config_base,
    device=device,
    ds_names=ds_names,
    densifiable_ds=densifiable_ds,
    attr_opt=True,
    plot_every=-1
    
)



In [ ]:
#TODO: GIT SHIT

# Ablation Study: Densification

In [ ]:

# ABLATION STUDY WITHOUT DENSIFICATION

model_names = ['pieclam']
ds_names = ['photo', 'elliptic', 'reddit']


final_accs = {'photo': {'vanilla_star':[],'prior':[],'prior_star':[]}, 'elliptic': {'vanilla_star':[],'prior':[],'prior_star':[]}, 'reddit': {'vanilla_star':[],'prior':[],'prior_star':[]}}

for model_name in model_names:    
    for ds_name in ds_names:
        
        ds = import_dataset(ds_name)
        for _ in range(10):   
            #! NOTICE: no densification!
            ds_to_use = ds

            
 
            # config_triplets = [['feat_opt', 'n_iter', 20],
            #                 ['prior_opt', 'n_iter', 20],]
            config_triplets = []

            trainer_anomaly = Trainer(
                model_name=model_name,
                device=device,
                dataset=ds_to_use.clone(),
                attr_opt=True,
                task='anomaly_unsupervised',
                use_global_configs_base=True,
                config_triplets_to_change=config_triplets
            )

            losses, acc_test, acc_val = trainer_anomaly.train(
                init_type='small_gaus',
                init_feats=True,
                acc_every=20,
                plot_every=10000,
                verbose=False,
                verbose_in_funcs=False
            )

            final_accs[ds_name]['vanilla_star'].append(acc_test['vanilla_star'][-1])
            final_accs[ds_name]['prior'].append(acc_test['prior'][-1])
            final_accs[ds_name]['prior_star'].append(acc_test['prior_star'][-1])

        if trainer_anomaly.clamiter.prior is not None:
            del trainer_anomaly.clamiter.prior.model
        del trainer_anomaly.data
        torch.cuda.empty_cache()

        del ds
        del ds_to_use
       
        torch.cuda.empty_cache()



In [ ]:
# get mean and std for final accuracies.
for key in final_accs:
    print(key)
    for key2 in final_accs[key]:
        print(key2, np.mean(final_accs[key][key2]), np.std(final_accs[key][key2]))

# Vanilla methods

In [ ]:
# anomaly detection with ieclam

model_names = ['ieclam']
ds_names = ['photo', 'elliptic', 'reddit']

for model_name in model_names:    
    for ds_name in ds_names:
        
        ds = import_dataset(ds_name)
        if ds_name in ['reddit', 'photo', 'elliptic']:
            fat_ds = TwoHop()(ds)
            fat_ds.edge_attr = torch.ones(fat_ds.edge_index.shape[1]).bool()
            ds_to_use = fat_ds
        
        # ds_to_use = ds
        losseses = []
        acc_testses = []
        acc_valses = []
        
        '''change some of the configs manually e.g. 
        config_triplets = [['feat_opt', 'n_iter', 1000], ['prior_opt, 'lr', 0.0001], ...]'''
        config_triplets = []

        trainer_anomaly = Trainer(
            model_name=model_name,
            device=device,
            dataset=ds_to_use.clone(),
            attr_opt=True,
            task='anomaly_unsupervised',
            global_config_base=True,
            config_triplets_to_change=config_triplets
        )

        losses, acc_test, acc_val = trainer_anomaly.train(
            init_type='small_gaus',
            init_feats=True,
            acc_every=20,
            plot_every=-1,
            verbose=True,
            verbose_in_funcs=False
        )
        losseses.append(losses)
        acc_testses.append(acc_test)
        acc_valses.append(acc_val)
        
        if trainer_anomaly.clamiter.prior is not None:
            del trainer_anomaly.clamiter.prior.model
        del trainer_anomaly.data
        torch.cuda.empty_cache()

    del ds
    del ds_to_use
    if ds_name in ['reddit', 'photo', 'elliptic']:
        del fat_ds
    torch.cuda.empty_cache()